In [2]:
# 225150200111004_1 HAIKAL THORIQ ATHAYA_1
# 225150200111008_2 MUHAMMAD ARSYA ZAIN YASHIFA_2
# 225150201111001_3 JAVIER AAHMES REANSYAH_3
# 225150201111003_4 MUHAMMAD HERDI ADAM_4

In [3]:
# Install dependencies as needed:
%pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the CSV file in the dataset
file_path = "tripadvisor_hotel_reviews.csv"

# Load the latest version
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "andrewmvd/trip-advisor-hotel-reviews",
    file_path,
)

print("First 5 records:")
display(df.head())

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


/Users/arsyazain/Downloads/SEMESTER ANTARA 7/MLOps/tripadvisor-sentiment-cicd/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/8m/4swyp5sx7gndw6f_287smdbw0000gn/T/ipykernel_75494/505985576.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


/Users/arsyazain/Downloads/SEMESTER ANTARA 7/MLOps/tripadvisor-sentiment-cicd/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/8m/4swyp5sx7gndw6f_287smdbw0000gn/T/ipykernel_75494/505985576.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:


,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [4]:
import os
os.makedirs('../Data', exist_ok=True)
df.to_csv('../Data/tripadvisor_hotel_reviews.csv', index=False)

In [5]:
# Preprocessing and training with TF-IDF and Logistic Regression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

df['Rating'] = df['Rating'].apply(lambda x: 1 if x >= 4 else 0)

X = df['Review']
y = df['Rating']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [7]:
# Train logistic regression model
model = LogisticRegression(max_iter=1000, solver='lbfgs')
model.fit(X_train_tfidf, y_train)

# Evaluate on evaluation dataset
y_pred = model.predict(X_test_tfidf)

# Print classification report
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

              precision    recall  f1-score   support

    Negative       0.88      0.74      0.80      1057
    Positive       0.91      0.96      0.94      3042

    accuracy                           0.91      4099
   macro avg       0.90      0.85      0.87      4099
weighted avg       0.90      0.91      0.90      4099



In [11]:
print(df['Review'].sample(1))

15055    okay..need communication pls, overall just oka...
Name: Review, dtype: object
